# Машинное обучение. Практическая работа

В этой практической работе четыре обязательные задачи.

Они помогут понять, что вы действительно усвоили материал модуля. 


Удачи!

## Цели практической работы

Научиться:
- обучать модели, основанные на деревьях решений;
- научиться оценивать и увеличивать их качество.

## Что входит в практическую работу

1. Загрузите датасет и ознакомьтесь с ним.
2. Подготовьте базовую модель дерева решений и измерьте её качество.
3. Подготовьте базовую модель случайного леса и измерьте её качество.
4. Увеличьте точность модели случайного леса на тестовых данных.
5. Проведите анализ влияния признаков на модель.

## Что оценивается

* Выполнены все четыре задачи. Для каждой:
 * в коде нет ручных перечислений, все действия автоматизированы;
 * результаты вычислений и применённых операций корректны;
 * ответы на вопросы, где требуется, корректны и обоснованы; 
 * код читабелен: переменным даны осмысленные названия, соблюдены отступы и правила расстановки пробелов; стилизация кода соответствует рекомендациям [PEP 8](https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html).

* Репозиторий проекта оформлен корректно:
 * содержит осмысленные коммиты, содержащие конкретные реализованные фичи;
 * ветки названы согласно назначению;
 * файлы, не связанные с проектом, не хранятся в репозитории.


## Как отправить работу на проверку

Сдайте практическую работу этого модуля через систему контроля версий Git сервиса Skillbox GitLab. После загрузки работы на проверку напишите об этом в личном кабинете своему куратору.

## Обязательные задачи

### Описание датасета:
- `id`— идентификатор записи;
- `is_manufacturer_name`— признак производителя автомобиля;

- `region_*`— регион;
- `x0_*`— тип топлива;
- `manufacturer_*`— производитель;
- `short_model_*`— сокращённая модель автомобиля;
- `title_status_*`— статус;
- `transmission_*`— коробка передач;
- `state_*`— штат;
- `age_category_*`— возрастная категория автомобиля;

- `std_scaled_odometer`— количество пройденных миль (после стандартизации);
- `year_std`— год выпуска (после стандартизации);
- `lat_std`— широта (после стандартизации);
- `long_std`— долгота (после стандартизации);
- `odometer/price_std`— отношение стоимости к пробегу автомобиля (после стандартизации);
- `desc_len_std`— количество символов в тексте объявления о продаже (после стандартизации);
- `model_in_desc_std`— количество наименований модели автомобиля в тексте объявления о продаже (после стандартизации);
- `model_len_std`— длина наименования автомобиля (после стандартизации);
- `model_word_count_std`— количество слов в наименовании автомобиля (после стандартизации);
- `month_std`— номер месяца размещения объявления о продаже автомобиля (после стандартизации);
- `dayofweek_std`— день недели размещения объявления о продаже автомобиля (после стандартизации);
- `diff_years_std`— количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля (после стандартизации);

- `price`— стоимость;
- `price_category`— категория цены.

1. Подготовка базовой модели

Обучите простую модель классификации с помощью DecisionTreeClassifier на данных из датасета vehicles_dataset_prepared.csv. Для этого сделайте шаги:

1. Обучите модель дерева решений с зафиксированным random_state на тренировочной выборке.
2. Сделайте предикт на тестовой выборке.
3. Замерьте точность на тестовой выборке и выведите матрицу ошибок. 
4. Удалите фичи с нулевыми весами по feature_importance из тренировочной и тестовой выборок.
5. Заново обучите модель и измерьте качество.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, confusion_matrix 
from sklearn.model_selection import GridSearchCV


In [31]:
df = pd.read_csv('data/vehicles_dataset_prepared.csv')

df_prepared = df.copy()
df_prepared = df_prepared.drop(['price', 'odometer/price_std'], axis=1)

x = df_prepared.drop(['price_category'], axis=1)
y = df_prepared['price_category']

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42
)
df.head()

,id,price,price_category,is_audi,is_ford,is_chevrolet,is_toyota,x0_diesel,x0_electric,x0_gas,...,long_std,year_std,odometer/price_std,desc_len_std,model_in_desc_std,model_len_std,model_word_count_std,month_std,dayofweek_std,diff_years_std
0,7308295377,54990,high,0,0,0,0,1.0,0.0,0.0,...,0.484245,1.322394,-0.510784,0.632075,-0.155788,1.163032,1.910669,-0.615846,1.120284,-1.322394
1,7316380095,16942,medium,0,1,0,0,0.0,0.0,0.0,...,1.110800,0.695973,-0.402947,-0.646781,-0.155788,0.932087,1.235799,1.623784,-1.374972,-0.695973
2,7313733749,35590,high,0,0,0,0,0.0,0.0,1.0,...,0.531185,0.852578,-0.514480,0.560744,-0.155788,0.470197,0.560930,-0.615846,-0.376870,-0.852578
3,7308210929,14500,medium,0,0,0,1,0.0,0.0,1.0,...,0.853562,0.226157,-0.241883,0.180435,-0.155788,-0.915473,-0.788810,-0.615846,1.120284,-0.226157
4,7303797340,14590,medium,0,0,0,0,0.0,0.0,0.0,...,0.557607,0.069552,-0.333074,0.766366,-0.155788,1.163032,1.910669,-0.615846,0.122182,-0.069552


In [3]:
#Обучите модель дерева решений с зафиксированным random_state на тренировочной выборке.
#Сделайте предикт на тестовой выборке.
#Замерьте точность на тестовой выборке и выведите матрицу ошибок.
#Удалите фичи с нулевыми весами по feature_importance из тренировочной и тестовой выборок.
#Заново обучите модель и измерьте качество

clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

predicted_train = clf.predict(x_train)
predicted_test = clf.predict(x_test)

print(accuracy_score(y_train, predicted_train))
print(accuracy_score(y_test, predicted_test))

y_pred = clf.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
f_imp_list = list(zip(x_train.columns, clf.feature_importances_))
f_imp_list.sort(key = lambda x: x[1], reverse=True)
f_imp_list

zero_weight_features = x_train.columns[clf.feature_importances_ == 0]
x_train_filtered = x_train.drop(zero_weight_features, axis=1)
x_test_filtered = x_test.drop(zero_weight_features, axis=1)

clf.fit(x_train_filtered, y_train)

y_pred_filtered = clf.predict(x_test_filtered)

accuracy_filtered = accuracy_score(y_test, y_pred_filtered)
conf_matrix_filtered = confusion_matrix(y_test, y_pred_filtered)

print(f"\nТочность после удаления фич с нулевыми весами: {accuracy_filtered}")
print("Матрица ошибок после удаления фич с нулевыми весами:")
print(conf_matrix_filtered)

1.0
0.6725571725571725
[[745  48 204]
 [ 43 683 227]
 [184 239 513]]

Точность после удаления фич с нулевыми весами: 0.6652806652806653
Матрица ошибок после удаления фич с нулевыми весами:
[[730  53 214]
 [ 57 674 222]
 [197 223 516]]


2. Подготовка модели случайного леса

Обучите простую модель классификации с помощью RandomForestClassifier. Для этого на новых урезанных семплах тренировочной и тестовой выборок обучите модель случайного леса с зафиксированным random_state=50. 

In [4]:
from sklearn.ensemble import RandomForestClassifier
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=50)

rf_clf = RandomForestClassifier()
rf_clf.fit(x_train, y_train)

predicted_train_rf = rf_clf.predict(x_train)
predicted_test_rf = rf_clf.predict(x_test)


2. Сделайте предикт, посчитайте точность модели и матрицу ошибок. Сравните с предыдущей моделью дерева решений. Есть ли случаи, когда модель из пункта 1 отрабатывает лучше, чем модель случайного леса?

In [5]:
# Ваш код здесь 
conf_matrix_random = confusion_matrix(y_test, predicted_test_rf)
accuracy = accuracy_score(y_test, predicted_test_rf)

print(accuracy_score(y_train, predicted_train_rf))
print(accuracy_score(y_test, predicted_test_rf))
print(f"\nТочность модели: {accuracy}");
print("Матрица ошибок:")
print(conf_matrix_random)

1.0
0.7595287595287595

Точность модели: 0.7595287595287595
Матрица ошибок:
[[787  46 113]
 [ 13 824 138]
 [164 220 581]]


3. Тюнинг модели случайного леса

Увеличьте точность модели на тестовом датасете RandomForestClassifier c помощью тюнинга параметров. 

Параметры, которые можно настраивать для увеличения точности:
```
    `bootstrap'
    'max_depth'
    'max_features'
    'min_samples_leaf'
    'min_samples_split'
    'random_state'
    'n_estimators'

```

С описанием каждого из параметров можно ознакомиться [в документации](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

Задание засчитывается, если значение метрики строго выше 0.76 на тестовом датасете.

In [54]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Grid Search
param_grid = {
   'n_estimators': list(range(100, 501, 100)),
   'max_features': ['sqrt', 'log2'],
   'min_samples_leaf': list(range(1, 10))
}
# Create RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=50)

# Create Grid Search
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the model to the data
grid_search.fit(x_train, y_train)

# Get the best parameters from the grid search
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Predictions on the test set using the best model
best_rf_model = grid_search.best_estimator_
predicted_test_rf_tuned = best_rf_model.predict(x_test)

# Calculate accuracy and confusion matrix for the tuned model
accuracy_tuned = accuracy_score(y_test, predicted_test_rf_tuned)
conf_matrix_tuned = confusion_matrix(y_test, predicted_test_rf_tuned)

# Print results
print(f"\nAccuracy of the tuned model: {accuracy_tuned}")
print("Confusion Matrix of the tuned model:")
print(conf_matrix_tuned)


Best Parameters: {'max_features': 'sqrt', 'min_samples_leaf': 5, 'n_estimators': 100}

Accuracy of the tuned model: 0.9996534996534997
Confusion Matrix of the tuned model:
[[996   0   1]
 [  0 953   0]
 [  0   0 936]]


4. Анализ влияния фичей на модель

До этого в задачах вы работали над подготовленным датасетом, где фичи были заранее извлечены из текстовых переменных, отскейлены и пропущены через OneHotEncoder. Сравним, какой была бы предсказательная способность модели, если бы мы использовали только сырые данные из исходного датасета. Для этого проделайте следующие шаги:

1. Загрузите датасет `vehicles_dataset_old.csv`.
2. Удалите из него переменную `price` и все строковые колонки. Дерево решений и случайный лес не умеют самостоятельно работать со строковыми значениями.
3. Сформируйте x_train и x_test так же, как они были сформированы в предыдущих заданиях.
4. Обучите свою лучшую модель случайного леса на новых данных и замерьте качество. Убедитесь, что оно ухудшилось.
5. Найдите три фичи, которые лучшим образом повлияли на предсказательную способность модели.

In [34]:
df_old = pd.read_csv('data/vehicles_dataset_old.csv')
df_old.head()

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,clean,other,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,tn,35.060000,-85.250000,2021-04-17T12:30:50-0400,high,2021-04-17 16:30:50+00:00
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,clean,automatic,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,nj,40.821805,-74.061962,2021-05-03T15:40:21-0400,medium,2021-05-03 19:40:21+00:00
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,clean,other,https://images.craigslist.org/00y0y_eeZjWeiSfb...,Carvana is the safer way to buy a car During t...,ca,33.779214,-84.411811,2021-04-28T03:52:20-0700,high,2021-04-28 10:52:20+00:00
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,clean,automatic,https://images.craigslist.org/00606_iGe5iXidib...,2013 Toyota RAV4 XLE 4dr SUV Offered by: R...,nc,35.715954,-78.655304,2021-04-17T10:08:57-0400,medium,2021-04-17 14:08:57+00:00
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,clean,other,https://images.craigslist.org/00F0F_5UAXmOzC18...,Carvana is the safer way to buy a car During t...,tn,35.970000,-83.940000,2021-04-08T15:10:56-0400,medium,2021-04-08 19:10:56+00:00


In [52]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Load the dataset
df_old = pd.read_csv('data/vehicles_dataset_old.csv')

# Drop the 'price' column and all string columns
df_old = df_old.drop(['id', 'url', 'region', 'region_url', 'manufacturer', 'model', 'fuel', 'title_status', 'transmission', 'image_url', 'description', 'state', 'posting_date', 'date'], axis=1)

# Separate target variable (price_category) from features
X_old = df_old.drop('price_category', axis=1)
y_old = df_old['price_category']

# Split the dataset into training and testing sets
x_train_old, x_test_old, y_train_old, y_test_old = train_test_split(X_old, y_old, test_size=0.2, random_state=42)

# Train your best random forest model on the new data
# Note: Replace RandomForestClassifier with your best-performing classification model
model_old = RandomForestClassifier()
model_old.fit(x_train_old, y_train_old)

# Make predictions on the test set
y_pred_old = model_old.predict(x_test_old)

# Evaluate the model performance
accuracy_old = accuracy_score(y_test_old, y_pred_old)
print(f'Accuracy: {accuracy_old}')

# Feature importance analysis
feature_importances_old = model_old.feature_importances_
feature_names_old = X_old.columns

# Sort features by importance
sorted_idx_old = feature_importances_old.argsort()[::-1]

# Display the top three most important features
print("Top 3 most important features:")
for i in range(3):
    print(f"{feature_names_old[sorted_idx_old[i]]}: {feature_importances_old[sorted_idx_old[i]]}")


Accuracy: 0.9994802494802495
Top 3 most important features:
price: 0.8397934503354194
year: 0.08989170504118618
odometer: 0.057670355832923625
